In [1]:
import os
print(os.getcwd())#显示当前路径
 
os.chdir('D:/bio/metirc/scib-main')#更改路径，''里面为更改的路径

print(os.getcwd())#显示当前路径

os.environ['R_HOME'] = "E:/R/R-4.2.1/"
os.environ['R_USER'] = "D:/anaconda/envs/index/Lib/site-packages/rpy2"

D:\bio\jupyter_code\index
D:\bio\metirc\scib-main


In [2]:
import index

# import importlib
# importlib.reload(index)

In [3]:
import scib
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad 

import esda
from esda.moran import Moran, Moran_Local
from esda.geary import Geary
from esda.geary_local import Geary_Local
from esda.getisord import G, G_Local

# load dataset

In [4]:
# adatas = sc.read_h5ad('G:/dataset/1_DLPFC/output/Donor1.h5ad')
# print(adatas)

In [5]:
# adatas.obs.to_csv('G:/dataset/1_DLPFC/output/Donor1_meta.csv')

In [4]:
import warnings
warnings.filterwarnings("ignore")

def load_data(method):
    adata_list = []
    for i in range(151673,151677):
        print(i)

        input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

        adata_temp = sc.read_visium(input_dir)
        adata_temp.var_names_make_unique()
        adata_temp.obs_names_make_unique()
        adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
        adata_temp.obs['batch'] = adata_label['batch'].astype("category")
        adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
        print(adata_temp)
        adata_list.append(adata_temp)

    adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151673,151677)])
    print(adata_pre)
    
    if method == 'harmony':
        harmony_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/Harmony/Donor1/embedding.csv")
        print(harmony_embedding_pd.shape)
        print(harmony_embedding_pd.head())
        adata_pre.obsm['harmony_embedding'] = harmony_embedding_pd.values
    if method == 'LIGER':
        LIGER_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/LIGER/Donor1/embedding.csv")
        print(LIGER_embedding_pd.shape)
        print(LIGER_embedding_pd.head())
        adata_pre.obsm['LIGER_embedding'] = LIGER_embedding_pd.values
    if method == 'DeepST': 
        DeepST_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/DeepST/Donor1/embedding.csv", index_col=0)
        print(DeepST_embedding_pd.shape)
        print(DeepST_embedding_pd.head())
        adata_pre.obsm['DeepST_embedding'] = DeepST_embedding_pd.values
        print(adata_pre)
    if method == 'STAGATE': 
        STAGATE_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/STAGATE/Donor3/embedding.csv")
        print(STAGATE_embedding_pd.shape)
        print(STAGATE_embedding_pd.head())
        adata_pre.obsm['STAGATE_embedding'] = STAGATE_embedding_pd.values
    return adata_pre

def cal_neighbor(adata_pre, my_methods):
    print(adata_pre)
    adatas = adata_pre
    # cal neighbors
    for method in my_methods:
        print(method)
        if 'GraphST' in method:
            sc.pp.neighbors(adatas, n_neighbors=50, use_rep= method+'_embedding', key_added = method+'_neighbors')
        else:
            sc.pp.neighbors(adatas, use_rep= method+'_embedding', key_added = method+'_neighbors')

    print(adatas)
    return adatas

def cal_cluster(adatas, my_methods):
    for method in my_methods:
        print(method)
        adatas.uns['neighbors'] = adatas.uns[method+'_neighbors'].copy()
        adatas.obsp['distances'] = adatas.obsp[method+'_neighbors_distances'].copy()
        adatas.obsp['connectivities'] = adatas.obsp[method+'_neighbors_connectivities'].copy()
        # leiden
        adatas = index.get_cluster_data(adatas, 7, True, 'leiden', 'leiden_'+method, resolution = sorted(list(np.arange(0.1, 1, 0.01)), reverse=False))
        # louvain
        adatas = index.get_cluster_data(adatas, 7, True, 'louvain', 'louvain_'+method, resolution = sorted(list(np.arange(0.1, 1, 0.01)), reverse=False))
        # mclust
        if method != 'GraphST3000':
            adatas = index.get_cluster_data(adatas, 7, True, 'mclust', 'mclust_'+method, None, method+'_embedding')
    
    return adatas

def cal_IsolatedLabelsF1(adatas, my_methods):
    import gc
    for method in my_methods:
        print(method)
        print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding"))
        gc.collect()
        print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="leiden_"+method, embed=method+"_embedding"))
        gc.collect()
        print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="louvain_"+method, embed=method+"_embedding"))
        gc.collect()
        print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="mclust_"+method, embed=method+"_embedding"))
        gc.collect()

def cal_PCR(adatas, my_methods):
    for method in my_methods:
        print(method)
        print(index.kBET(adatas, batch_key="batch", label_key='ground.truth', type_="embed", embed=method+"_embedding"))
        print(index.kBET(adatas, batch_key="batch", label_key='leiden_'+method, type_="embed", embed=method+"_embedding"))
        print(index.kBET(adatas, batch_key="batch", label_key='louvain_'+method, type_="embed", embed=method+"_embedding"))
        print(index.kBET(adatas, batch_key="batch", label_key='mclust_'+method, type_="embed", embed=method+"_embedding"))
        

# for method in ['harmony', 'LIGER', 'STAGATE', 'DeepST']:
#     adata_pre = load_data(method)
#     adatas = cal_neighbor(adata_pre, [method])
#     adatas = cal_cluster(adatas, [method])
#     cal_IsolatedLabelsF1(adatas, [method])
#     cal_PCR(adatas, [method])

In [5]:
adata_pre = load_data('STAGATE')

151673
AnnData object with n_obs × n_vars = 3639 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151674
AnnData object with n_obs × n_vars = 3673 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151675
AnnData object with n_obs × n_vars = 3592 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151676
AnnData object with n_obs × n_vars = 3460 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
AnnData object with n_obs × n_vars = 14364 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    obsm: 'spatia

In [7]:
adatas = adata_pre
method = 'STAGATE'
print(index.cASW(adatas, 'ground.truth', method+'_embedding'))
print(index.cLISI(adatas, 'ground.truth', type_="embed", use_rep=method+'_embedding'))
print(index.IsolatedLabelsAsw(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding"))
print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding"))
print(index.iASW(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding", type_="embed"))


0.5499411341845908
0.9601123100495658
isolated labels: no more than 4 batches per label
Layer 1: 0.5990493594042564
Layer 2: 0.51711831164603
Layer 3: 0.5080254035230254
Layer 4: 0.5275695248104225
Layer 5: 0.5342755208062423
Layer 6: 0.45851789557207817
WM: 0.7301397493629744
na: 0.4522655028859638
0.5408701585013742
isolated labels: no more than 4 batches per label
Compute neighbors on rep STAGATE_embedding
Layer 1: 0.9144587433762301
Compute neighbors on rep STAGATE_embedding
Layer 2: 0.3937099592312172
Compute neighbors on rep STAGATE_embedding
Layer 3: 0.7018097898700352
Compute neighbors on rep STAGATE_embedding
Layer 4: 0.3242467718794835
Compute neighbors on rep STAGATE_embedding
Layer 5: 0.6444246737841043
Compute neighbors on rep STAGATE_embedding
Layer 6: 0.6020777965692196
Compute neighbors on rep STAGATE_embedding
WM: 0.9182973572725439
Compute neighbors on rep STAGATE_embedding
na: 0.23052959501557632
0.5911943358748012
mean silhouette per group:          silhouette_score

#### load origin dataset  &  embedding  &  coordinates

In [4]:
adata_list = []
for i in range(151507,151511):
    print(i)

    input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

    adata_temp = sc.read_visium(input_dir)
    adata_temp.var_names_make_unique()
    adata_temp.obs_names_make_unique()
    adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
    adata_temp.obs['batch'] = adata_label['batch'].astype("category")
    adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
    print(adata_temp)
    adata_list.append(adata_temp)

adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151507,151511)])
print(adata_pre)

del adata_temp
del adata_list

151507


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151508


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4384 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151509


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4789 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151510


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4634 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
AnnData object with n_obs × n_vars = 18033 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    obsm: 'spatial'


In [5]:
my_methods = ['seurat']

In [6]:
# seurat embedding (18033, 50)  (15284, 50)  (14364, 50)
seurat_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/Seurat/Donor1/seurat.PCs.tsv")
print(seurat_embedding_pd.shape)
print(seurat_embedding_pd.head())
adata_pre.obsm['seurat_embedding'] = seurat_embedding_pd.values

(18033, 50)
                           PC_1       PC_2      PC_3      PC_4      PC_5  \
AAACAACGAATAGTTC-1_1   2.430077   4.386316 -3.432069  4.971501 -5.658500   
AAACAAGTATCTCCCA-1_1  -5.961436  -7.011905  0.824565 -1.364521  0.645884   
AAACAATCTACTAGCA-1_1   6.975747  11.326056  4.891646  3.530083  4.149356   
AAACACCAATAACTGC-1_1  56.483353 -21.536790  5.328702 -6.632643  3.528389   
AAACAGCTTTCAGAAG-1_1   0.701552   4.008525 -4.675477  2.337909 -8.831374   

                           PC_6      PC_7      PC_8      PC_9     PC_10  ...  \
AAACAACGAATAGTTC-1_1  -0.294802 -2.123840 -1.945736  1.967007 -0.577177  ...   
AAACAAGTATCTCCCA-1_1  -1.043958 -2.973289  1.532887  0.836794  0.207672  ...   
AAACAATCTACTAGCA-1_1  -1.065639 -3.455331  1.959479  5.690728 -1.415511  ...   
AAACACCAATAACTGC-1_1  -4.920774  0.738674  0.358782 -0.493815  4.256827  ...   
AAACAGCTTTCAGAAG-1_1 -10.896230 -1.991882  0.943105  2.246593  1.469252  ...   

                         PC_41     PC_42     PC_43

In [9]:
# # harmony embedding (18033, 50)  (15284, 50)  (14364, 50)
# harmony_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/Harmony/Donor2/embedding.csv")
# print(harmony_embedding_pd.shape)
# print(harmony_embedding_pd.head())
# adata_pre.obsm['harmony_embedding'] = harmony_embedding_pd.values

In [10]:
# # LIGER　embedding  (18033, 20) (15284, 20) (14364, 20)
# LIGER_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/LIGER/Donor2/embedding.csv")
# print(LIGER_embedding_pd.shape)
# print(LIGER_embedding_pd.head())
# adata_pre.obsm['LIGER_embedding'] = LIGER_embedding_pd.values

In [11]:
# # PRECAST　embedding  (18026, 15)  (15282, 15)  (14364, 15)
# PRECAST_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/PRECAST/Donor2/embedding.csv")
# print(PRECAST_embedding_pd.shape)
# print(PRECAST_embedding_pd.head())
# print(adata_pre.obs_names[:5])
# PRECAST_embedding_pd.rename(index=lambda s: s[:len(s)-1]+'-'+str(int(s[-1])+151668), inplace=True)
# print(PRECAST_embedding_pd.head())

# ######### 过滤细胞  取子集anndata
# adata_pre = adata_pre[PRECAST_embedding_pd.index.to_list(), :]
# adata_pre.obsm['PRECAST_embedding'] = PRECAST_embedding_pd.values

# ######### 填充embedding  fillna -- 0 
# # df_temp = pd.merge(adata_pre.obs['batch'],PRECAST_embedding_pd,how='left', left_index=True, right_index=True)
# # df_temp.drop(columns=['batch'], inplace=True)
# # df_temp.fillna(0, inplace=True)
# # print(df_temp.head())
# # adata_pre.obsm['PRECAST_embedding'] = df_temp.values
# # del df_temp

# print(adata_pre)

In [12]:
# # STAligner embedding (17985, 30)   (15101, 30)  (14243, 30)
# STAligner_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/STAligner/Donor2/embedding.csv")
# print(STAligner_embedding_pd.shape)
# print(STAligner_embedding_pd.head())
# print(adata_pre.obs_names[:5])
# STAligner_embedding_pd.rename(index=lambda s: s.replace("_", "-"), inplace=True)

# ######### 过滤细胞  取子集anndata
# adata_pre = adata_pre[STAligner_embedding_pd.index.to_list(), :]
# adata_pre.obsm['STAligner_embedding'] = STAligner_embedding_pd.values

# ######### 填充embedding  fillna -- 0 
# # df_temp = pd.merge(adata_pre.obs['batch'],STAligner_embedding_pd,how='left', left_index=True, right_index=True)
# # df_temp.drop(columns=['batch'], inplace=True)
# # df_temp.fillna(0, inplace=True)
# # print(df_temp.shape)
# # print(df_temp.head())
# # adata_pre.obsm['STAligner_embedding'] = df_temp.values
# # del df_temp

# print(adata_pre)

In [13]:
# # DeepST embedding  (18033, 28)  (15284, 28)  (14364, 28)
# DeepST_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/DeepST/Donor1/embedding.csv", index_col=0)
# print(DeepST_embedding_pd.shape)
# print(DeepST_embedding_pd.head())
# adata_pre.obsm['DeepST_embedding'] = DeepST_embedding_pd.values
# print(adata_pre)

In [7]:
# # GraphST embedding    (14243, 3000)
# GraphST_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/GraphST/GraphST_mat_151669_151670_151671_151672.txt", index_col=0)
# print(GraphST_embedding_pd.shape)
# print(GraphST_embedding_pd.head())

# GraphST_embedding_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
# # ######### 过滤细胞  取子集anndata
# adata_pre = adata_pre[GraphST_embedding_pd.index.to_list(), :]
# adata_pre.obsm['GraphST3000_embedding'] = GraphST_embedding_pd.values

# # ######### 填充embedding  fillna -- 0 
# # df_temp = pd.merge(adata_pre.obs['batch'], GraphST_embedding_pd, how='left', left_index=True, right_index=True)
# # df_temp.drop(columns=['batch'], inplace=True)
# # df_temp.fillna(0, inplace=True)
# # print(df_temp.shape)
# # adata_pre.obsm['GraphST_embedding'] = df_temp.values
# # del df_temp

# from sklearn.decomposition import PCA
# pca = PCA(n_components=20, random_state=42)  # code from GraphST  https://github.com/JinmiaoChenLab/GraphST/blob/main/GraphST/utils.py
# embedding = pca.fit_transform(adata_pre.obsm['GraphST3000_embedding'].copy())
# adata_pre.obsm['GraphST20_embedding'] = embedding

# del GraphST_embedding_pd
# del embedding

# print(adata_pre)

(15101, 3000)
                           GraphST-0  GraphST-1  GraphST-2  GraphST-3  \
151669-AAACAAGTATCTCCCA-1   0.886127   0.182383   0.238874   0.327230   
151669-AAACACCAATAACTGC-1   0.378642   0.091632   0.171001   0.031319   
151669-AAACAGAGCGACTCCT-1   0.637213   0.048262   0.159262   0.119652   
151669-AAACAGGGTCTATATT-1   0.761914   0.341203   0.338071  -0.041657   
151669-AAACATTTCCCGGATT-1   0.625582   0.099525   0.267691   0.393116   

                           GraphST-4  GraphST-5  GraphST-6  GraphST-7  \
151669-AAACAAGTATCTCCCA-1   0.380450   0.124853   0.160642   0.095046   
151669-AAACACCAATAACTGC-1   0.044300   0.068142  -0.037427   0.624840   
151669-AAACAGAGCGACTCCT-1   0.158152  -0.037258   0.294591   0.007088   
151669-AAACAGGGTCTATATT-1   0.105920   0.115709   0.064444   0.321755   
151669-AAACATTTCCCGGATT-1   0.356661   0.160610   0.366360   0.332868   

                           GraphST-8  GraphST-9  ...  GraphST-2990  \
151669-AAACAAGTATCTCCCA-1   0.040188  

In [6]:
# # STAGATE embedding (18033, 30)  (15284, 30)  (14364, 30)
# STAGATE_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/STAGATE/Donor1/embedding.csv")
# print(STAGATE_embedding_pd.shape)
# print(STAGATE_embedding_pd.head())
# adata_pre.obsm['STAGATE_embedding'] = STAGATE_embedding_pd.values

(18033, 30)
                           STAGATE_1  STAGATE_2  STAGATE_3  STAGATE_4  \
AAACAACGAATAGTTC-1_151507  -4.345402   1.667365   1.845119   5.311783   
AAACAAGTATCTCCCA-1_151507  -5.505312   2.322926   0.794274   9.065916   
AAACAATCTACTAGCA-1_151507  -5.409313  -0.734618   0.204319   3.340324   
AAACACCAATAACTGC-1_151507  -2.703301   0.738970   2.781152   2.334350   
AAACAGCTTTCAGAAG-1_151507  -6.101233   2.152689   3.326889   8.605307   

                           STAGATE_5  STAGATE_6  STAGATE_7  STAGATE_8  \
AAACAACGAATAGTTC-1_151507  -2.591382  -0.538832   5.007315   1.513464   
AAACAAGTATCTCCCA-1_151507  -2.857811  -0.160081   4.021773   1.688442   
AAACAATCTACTAGCA-1_151507  -1.626888  -0.530949   3.794722   0.325293   
AAACACCAATAACTGC-1_151507  -1.128294   1.648088  -1.296323   0.788101   
AAACAGCTTTCAGAAG-1_151507  -3.238205  -0.217724   4.519947   1.033965   

                           STAGATE_9  STAGATE_10  ...  STAGATE_21  STAGATE_22  \
AAACAACGAATAGTTC-1_151507   0

In [16]:
# SPIRAL embedding  (17985, 32)  (15101, 32)
# SPIRAL_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/output/spiral/SPIRAL_151673_151674_151675_151676_embed_512.csv", index_col=0)
# print(SPIRAL_embedding_pd.shape)
# print(SPIRAL_embedding_pd.iloc[:,:5].head())
# print(adata_pre.obs_names[:5])
# SPIRAL_embedding_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
# adata_pre = adata_pre[SPIRAL_embedding_pd.index.to_list(), :]
# adata_pre.obsm['SPIRAL_embedding'] = SPIRAL_embedding_pd.values
# print(adata_pre)


# SPIRAL_pyg_embedding_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/output/SPIRAL_pyg_151673_151674_151675_151676_embed.csv", index_col=0)
# print(SPIRAL_pyg_embedding_pd.shape)
# print(SPIRAL_pyg_embedding_pd.iloc[:,:5].head())
# print(adata_pre.obs_names[:5])
# SPIRAL_pyg_embedding_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
# adata_pre = adata_pre[SPIRAL_pyg_embedding_pd.index.to_list(), :]
# adata_pre.obsm['SPIRAL_pyg_embedding'] = SPIRAL_pyg_embedding_pd.values
# print(adata_pre)

(14243, 32)
                           SPIRAL_0  SPIRAL_1  SPIRAL_2  SPIRAL_3  SPIRAL_4
151673-AAACAAGTATCTCCCA-1  0.017159  0.506587  0.233944  0.173340  0.109793
151673-AAACAATCTACTAGCA-1 -0.071980  0.415265 -0.235092  0.021972  0.080909
151673-AAACACCAATAACTGC-1 -0.422182 -0.058848  0.012611  0.045996 -0.064233
151673-AAACAGAGCGACTCCT-1  0.395679  0.555186  0.188891 -0.065386 -0.136478
151673-AAACAGCTTTCAGAAG-1  0.049217 -0.510062  0.167246  0.193913  0.410749
Index(['AAACAAGTATCTCCCA-1-151673', 'AAACAATCTACTAGCA-1-151673',
       'AAACACCAATAACTGC-1-151673', 'AAACAGAGCGACTCCT-1-151673',
       'AAACAGCTTTCAGAAG-1-151673'],
      dtype='object')
AnnData object with n_obs × n_vars = 14243 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    obsm: 'spatial', 'SPIRAL_pyg_embedding'


In [17]:
# # SPIRAL embedding  (18033, 32)  (15284, 32)  (14364, 32)  (47329, 32)
# SPIRAL_embedding1024_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/SPIRAL_151507_151508_151509_151510_151669_151670_151671_151672_151673_151674_151675_151676_embed_1024.csv", index_col=0)
# print(SPIRAL_embedding1024_pd.shape)
# print(SPIRAL_embedding1024_pd.head())

# df = pd.DataFrame(data=None)
# for i in range(151507,151511):
#     print(i)
#     df_temp = SPIRAL_embedding1024_pd[SPIRAL_embedding1024_pd.index.str.contains(str(i))]
#     df = pd.concat([df, df_temp])
# print(df.head())
# SPIRAL_embedding1024_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
# df_temp = pd.merge(adata_pre.obs['batch'], SPIRAL_embedding1024_pd, how='left', left_index=True, right_index=True)
# df_temp.drop(columns=['batch'], inplace=True)
# df_temp.fillna(0, inplace=True)
# print(df_temp.shape)
# adata_pre.obsm['SPIRAL1024_embedding'] = df_temp.values

# # (15284, 32)
# SPIRAL_embedding2048_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/SPIRAL_pyg_151507_151508_151509_151510_151669_151670_151671_151672_151673_151674_151675_151676_embed_2048.csv", index_col=0)
# print(SPIRAL_embedding2048_pd.shape)
# print(SPIRAL_embedding2048_pd.head())

# df = pd.DataFrame(data=None)
# for i in range(151507,151511):
#     print(i)
#     df_temp = SPIRAL_embedding2048_pd[SPIRAL_embedding2048_pd.index.str.contains(str(i))]
#     df = pd.concat([df, df_temp])
# print(df.head())
# SPIRAL_embedding2048_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
# df_temp = pd.merge(adata_pre.obs['batch'], SPIRAL_embedding2048_pd, how='left', left_index=True, right_index=True)
# df_temp.drop(columns=['batch'], inplace=True)
# df_temp.fillna(0, inplace=True)
# print(df_temp.shape)
# adata_pre.obsm['SPIRAL2048_embedding'] = df_temp.values
# del df_temp
# del df

In [18]:
# # STUtility coor  (17889, 8)
# STUtility_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/STUtility/Donor1-STutil-coords.csv", sep=" ")
# print(STUtility_coor_pd.shape)
# print(STUtility_coor_pd.head())
# STUtility_coor_pd.rename(index=lambda s: s.split('_',1)[0] + '-' + str(int(s.split('_',1)[1]) + 151506), inplace=True)
# print(STUtility_coor_pd.head())
# print(STUtility_coor_pd.tail())
# adatas = adata_pre[STUtility_coor_pd.index.tolist(), :]
# adatas.obsm['spatial_aligned'] = STUtility_coor_pd[['align_x', 'align_y']].values
# adatas.obsm['spatial_origin'] = STUtility_coor_pd[['x', 'y']].values
# print(adatas)

In [19]:
# # moscot  (18033, 2)
# # moscot_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/moscot/moscot_affine_donor1.txt", header = None)
# moscot_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/moscot/moscot_warp_donor1.txt", header = None)
# print(moscot_coor_pd.shape)
# print(moscot_coor_pd.head())

# adatas = adata_pre

# adatas.obsm['spatial_aligned'] = moscot_coor_pd.values
# adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()

# print(adatas)

In [20]:
# # STalign 
# adata_list = []
# for i in range(151507,151511):
#     print(i)

#     input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

#     adata_temp = sc.read_visium(input_dir)
#     adata_temp.var_names_make_unique()
#     adata_temp.obs_names_make_unique()
#     adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
#     adata_temp.obs['batch'] = adata_label['batch'].astype("category")
#     adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
#     if i != 151510:
#         STalign_coor_pd = pd.read_csv('G:/dataset/1_DLPFC/output/STalign/'+str(i)+'_aligned_to_'+str(i+1)+'.csv')
#         STalign_coor_pd.index = STalign_coor_pd['0']
#         adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
#         adata_temp.obsm['spatial_aligned'] = STalign_coor_pd[['aligned_x', 'aligned_y']].values
#     else:
#         adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
#         adata_temp.obsm['spatial_aligned'] = adata_temp.obsm['spatial'].copy()
    
#     print(adata_temp)
#     adata_list.append(adata_temp)

# adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151507,151511)])
# print(adata_pre)

# del adata_temp
# del adata_list

In [21]:
# # paste  17978 × 33538
# adata_list = []
# for i in range(151507,151511):
#     print(i)

#     input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

#     adata_temp = sc.read_visium(input_dir)
#     adata_temp.var_names_make_unique()
#     adata_temp.obs_names_make_unique()
#     adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
#     adata_temp.obs['batch'] = adata_label['batch'].astype("category")
#     adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
    
#     paste_coor_pd = pd.read_csv('G:/dataset/1_DLPFC/output/PASTE/'+str(i)+'.csv')
#     paste_coor_pd.index = paste_coor_pd['spot']
#     paste_coor_pd.rename(index=lambda s: s.split('.',1)[0], inplace=True)
#     adata_temp = adata_temp[paste_coor_pd.index.tolist(), :]
#     adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
#     adata_temp.obsm['spatial_aligned'] = paste_coor_pd[['aligned_x', 'aligned_y']].values
    
#     print(adata_temp)
#     adata_list.append(adata_temp)

# adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151507,151511)])
# print(adata_pre)

# del adata_temp
# del adata_list

In [22]:
# # SPACEL  (18033, 3)
# SPACEL_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPACEL/donor1_aligned_coordinates.csv")
# print(SPACEL_coor_pd.shape)
# print(SPACEL_coor_pd.head())

# adatas = adata_pre

# adatas.obsm['spatial_aligned'] = SPACEL_coor_pd[['X','Y']].values
# adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()

# print(adatas)

In [23]:
# # SLAT
# adata = sc.read_h5ad('G:/dataset/1_DLPFC/output/SLAT/SLAT_aligned_151507.h5ad')
# print(adata)

In [24]:
# #SPIRAL  (47329, 6)  (17985, 6)
# SPIRAL_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/new_coord_151507_151508_151509_151510_151669_151670_151671_151672_151673_151674_151675_151676_modify.csv")
# print(SPIRAL_coor_pd.shape)
# print(SPIRAL_coor_pd.head())
# SPIRAL_coor_pd = SPIRAL_coor_pd[SPIRAL_coor_pd['batch']<=151510]
# print(SPIRAL_coor_pd.shape)
# SPIRAL_coor_pd.index = SPIRAL_coor_pd['Unnamed: 0']
# SPIRAL_coor_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
# print(SPIRAL_coor_pd.head())
# adatas = adata_pre[SPIRAL_coor_pd.index.tolist(), :]
# adatas.obsm['spatial_aligned'] = SPIRAL_coor_pd[['x', 'y']].values
# adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()
# print(adatas)

#### load integrated dataset

In [25]:
# adata_int = sc.read_h5ad('G:/dataset/1_DLPFC/output/Seurat/Donor1/Donor1.h5ad')
# print(adata_int)

# adata_int.obs.drop(['integrated_snn_res.'+str(round(i,2)) for i in np.arange(0.15, 0.51, 0.01)], axis=1, inplace=True)

# print(adata_int)

#### calculate cluster

In [7]:
print(adata_pre)
adatas = adata_pre
# methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'GraphST20', 'GraphST3000', 'SPIRAL1024', 'SPIRAL2048']
# cal neighbors
for method in my_methods:
    print(method)
    if 'GraphST' in method:
        sc.pp.neighbors(adatas, n_neighbors=50, use_rep= method+'_embedding', key_added = method+'_neighbors')
    else:
        sc.pp.neighbors(adatas, use_rep= method+'_embedding', key_added = method+'_neighbors')
    
print(adatas)

AnnData object with n_obs × n_vars = 18033 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    obsm: 'spatial', 'seurat_embedding'
seurat
AnnData object with n_obs × n_vars = 18033 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    uns: 'seurat_neighbors'
    obsm: 'spatial', 'seurat_embedding'
    obsp: 'seurat_neighbors_distances', 'seurat_neighbors_connectivities'


In [14]:
adatas.obs['ground.truth'].value_counts()

Layer 3    6258
Layer 1    4052
Layer 5    2085
Layer 2    1852
Layer 4    1429
Layer 6    1405
WM          904
na           48
Name: ground.truth, dtype: int64

In [8]:
for method in my_methods:
    print(method)
    adatas.uns['neighbors'] = adatas.uns[method+'_neighbors'].copy()
    adatas.obsp['distances'] = adatas.obsp[method+'_neighbors_distances'].copy()
    adatas.obsp['connectivities'] = adatas.obsp[method+'_neighbors_connectivities'].copy()
    # leiden
    adatas = index.get_cluster_data(adatas, 7, True, 'leiden', 'leiden_'+method, resolution = sorted(list(np.arange(0.1, 1, 0.01)), reverse=False))
    # louvain
    adatas = index.get_cluster_data(adatas, 7, True, 'louvain', 'louvain_'+method, resolution = sorted(list(np.arange(0.1, 1, 0.01)), reverse=False))
    # mclust
    if method != 'GraphST3000':
        adatas = index.get_cluster_data(adatas, 7, True, 'mclust', 'mclust_'+method, None, method+'_embedding')


seurat
resolution:  0.1    count_unique_leiden:  5
resolution:  0.11    count_unique_leiden:  4
resolution:  0.12    count_unique_leiden:  5
resolution:  0.13    count_unique_leiden:  5
resolution:  0.13999999999999999    count_unique_leiden:  5
resolution:  0.14999999999999997    count_unique_leiden:  5
resolution:  0.15999999999999998    count_unique_leiden:  5
resolution:  0.16999999999999998    count_unique_leiden:  5
resolution:  0.17999999999999997    count_unique_leiden:  6
resolution:  0.18999999999999995    count_unique_leiden:  6
resolution:  0.19999999999999996    count_unique_leiden:  6
resolution:  0.20999999999999996    count_unique_leiden:  6
resolution:  0.21999999999999995    count_unique_leiden:  7
Best resolution:  0.21999999999999995
resolution:  0.1    count_unique_leiden:  3
resolution:  0.11    count_unique_leiden:  4
resolution:  0.12    count_unique_leiden:  5
resolution:  0.13    count_unique_leiden:  5
resolution:  0.13999999999999999    count_unique_leiden: 

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.0.0
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |                                                                      |   0%

D:\anaconda\envs\index\lib\site-packages\rpy2\robjects\numpy2ri.py:252: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '


  |======================================================================| 100%


In [9]:
print(adatas)

AnnData object with n_obs × n_vars = 18033 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth', 'leiden', 'leiden_seurat', 'louvain', 'louvain_seurat', 'mclust_seurat'
    uns: 'seurat_neighbors', 'neighbors', 'leiden', 'leiden_seurat_resolution', 'louvain', 'louvain_seurat_resolution'
    obsm: 'spatial', 'seurat_embedding'
    obsp: 'seurat_neighbors_distances', 'seurat_neighbors_connectivities', 'distances', 'connectivities'


In [29]:
# adatas.obs.drop(['integrated_snn_res.0.5'], axis=1, inplace=True)
# print(adatas)

In [30]:
# adata_int.obs.drop(['integrated_snn_res.'+str(round(i,2)) for i in np.arange(0.11, 0.5, 0.01)], axis=1, inplace=True)
# print(adata_int.uns['leiden'])
# print(adata_int.uns['leiden_seurat_resolution'])

In [31]:
# print(adatas.isbacked)
# adatas.filename = 'G:/dataset/1_DLPFC/output/Donor1.h5ad'
# print(adatas.isbacked)

# Spatial index

In [32]:
# 指标：  batch混合程度 + 空间信息

# 两个细胞  局部空间

# print(index.cLISI(adatas, 'ground.truth', type_="knn", use_rep='spatial'))

# 去完批次   低维空间相近  ->  空间距离是否相近

# embedding knn:  1  ->    2 3 4
# spatial knn:    1  ->      3 4 5


#### spatial_coherence_score

In [14]:
# code from SPIRAL 
# https://github.com/guott15/SPIRAL/blob/main/Downstream/metrics_spatial_coherence.ipynb
# adatas.obs['leiden_seurat']=np.array(clust.loc[adata.obs_names,:].values,dtype=str)
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']
methods = []
my_methods = ['GraphST20']
for method in my_methods:
    methods.append('leiden_'+method)
    methods.append('louvain_'+method)
    methods.append('mclust_'+method)
print(methods)

for method in methods:
    print(method)
    knn = 6
    g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs[method] ,knn)
    # scs = index.spatial_entropy(g, node_dict)
    scs = index.spatial_coherence_score(g, node_dict, 100)
    print(scs)
    adatas.uns['scs_' + method] = scs


['leiden_GraphST20', 'louvain_GraphST20', 'mclust_GraphST20']
leiden_GraphST20
2.0455699624631842
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
-277.84348574372
louvain_GraphST20
2.086382721774155
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
-271.67625420137267
mclust_GraphST20
2.1866940273856734
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
7

In [15]:
for method in methods:
    print(adatas.uns['scs_' + method])

-277.84348574372
-271.67625420137267
-282.95259909700457


In [35]:
# # for aligned coor

# # source
# adatas.obsm['spatial'] = adatas.obsm['spatial_origin'].copy()
# knn = 6
# g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs['ground.truth'] ,knn)
# scs = index.spatial_entropy(g, node_dict)
# print(scs)


# # align
# adatas.obsm['spatial'] = adatas.obsm['spatial_aligned'].copy()
# knn = 6
# g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs['ground.truth'] ,knn)
# scs = index.spatial_entropy(g, node_dict)
# print(scs)


#### Moran's I

In [36]:
# from libpysal.weights import W
# from esda.moran import Moran, Moran_Local
# matrix = adatas.obsp['connectivities'].toarray()
# print(np.array_equal(matrix, matrix.T))
# neighbors = dict()
# val = dict()
# for i in range(0, adatas.n_obs):
#     c = list()
#     d = list()
#     for j in range(0, adatas.n_obs):
#         if matrix[i][j] != 0:
#             c.append(j)
#             d.append(matrix[i][j])
#     neighbors[i] = c
#     val[i] = d
# del matrix
# del val

# y = adatas.obsm['seurat_embedding'].T[0]
# # neighbors = W(neighbors)
# mi = Moran(y, neighbors)    # y:ndarray  value to analysis        neighbors:dict     weights:dict  Key is region ID, value is a list of edge weights.
# mil = Moran_Local(y, neighbors)    # scipy.sparse matrix object
# # mi = index.Moran_I(adatas.obsm['seurat_embedding'].T[1], neighbors)


# print(type[mi])
# print("Moran's I 值为：", mi.I)
# print("随机分布假设下Z检验值为：", mi.z_rand)
# print("随机分布假设下Z检验的P值为：", mi.p_rand)
# print("正态分布假设下Z检验值为：", mi.z_norm)
# print("正态分布假设下Z检验的P值为：", mi.p_norm)
# print(type[mil])
# # print("(if permutations>0) values indicate quandrant location 1 HH, 2 LH, 3 LL, 4 HL:", mil.q)
# # print("(if permutations>0) p-values based on standard normal approximation from permutations (one-sided) for two-sided tests, these values should be multiplied by 2:", mil.p_z_sim)

In [37]:

# # 指定空间基因

# # 找空间基因  SPARK  （500、1000、10%、20%）

# # 按batch分开

# # SPAGCN   找 SVG    12 --- 1000+        

# pca50 --> moran' I

# # source
# adatas.obsm['spatial'] = adatas.obsm['spatial_origin'].copy()
# sc.pp.neighbors(adatas)
# mi_origin = sc.metrics.morans_i(adatas)
# print(mi_origin)
# print(np.nanmean(mi_origin))



# adatas.obsm['spatial'] = adatas.obsm['spatial_aligned'].copy()
# sc.pp.neighbors(adatas)
# mi_align = sc.metrics.morans_i(adatas)
# print(mi_align)
# print(np.nanmean(mi_align))



In [38]:
for method in my_methods:
    print(method + ':')
    adatas.uns['neighbors'] = adatas.uns[method+'_neighbors'].copy()
    adatas.obsp['distances'] = adatas.obsp[method+'_neighbors_distances'].copy()
    adatas.obsp['connectivities'] = adatas.obsp[method+'_neighbors_connectivities'].copy()
    print(np.mean(sc.metrics.morans_i(adatas, obsm = method+'_embedding')))

# for method in my_methods:
#     print(method + ':')
#     print(np.mean(sc.metrics.morans_i(adatas, obsm = method+'_embedding')))

SPIRAL_pyg:
0.7048270055773314


#### Geary's C

In [39]:
# from libpysal.weights import W
# from esda.geary import Geary
# from esda.geary_local import Geary_Local


# w = W(neighbors)
# gc = Geary(y, w)
# gcl = Geary_Local(connectivity)    # scipy.sparse matrix object

# print(gc)

In [40]:
# # print(sc.metrics.gearys_c(adatas, obsm="seurat_embedding"))
# # print(sc.metrics.gearys_c(adatas.obsp["connectivities"], adatas.obsm["seurat_embedding"].T))


# adatas.obsm['spatial'] = adatas.obsm['spatial_origin'].copy()
# sc.pp.neighbors(adatas)
# gc_origin = sc.metrics.gearys_c(adatas)
# print(gc_origin)
# print(np.nanmean(gc_origin))

# adatas.obsm['spatial'] = adatas.obsm['spatial_aligned'].copy()
# sc.pp.neighbors(adatas)
# gc_align = sc.metrics.gearys_c(adatas)
# print(gc_align)
# print(np.nanmean(gc_align))



In [41]:
for method in my_methods:
    print(method + ':')
    adatas.uns['neighbors'] = adatas.uns[method+'_neighbors'].copy()
    adatas.obsp['distances'] = adatas.obsp[method+'_neighbors_distances'].copy()
    adatas.obsp['connectivities'] = adatas.obsp[method+'_neighbors_connectivities'].copy()
    print(np.mean(sc.metrics.gearys_c(adatas, obsm = method+'_embedding')))

SPIRAL_pyg:
0.25482821430783753


#### SSIM

In [42]:
# groups = adatas.obs.groupby("batch").indices
# adata_list = [adatas[i] for i in groups.values()]

# for i in range(len(adata_list)):
#     adata_list[i].obsm['spatial_aligned'] = pd.DataFrame(adata_list[i].obsm['spatial_aligned'], index=adata_list[i].obs_names)
#     adata_list[i].obsm['spatial_aligned'].columns = ['X', 'Y']

# # each slice is rasterized on the overlap of the previous slice
# for i in range(1, len(adata_list)):
#     x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i - 1].obsm['spatial_aligned'].max().X]))
#     y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i - 1].obsm['spatial_aligned'].max().Y]))
#     x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i - 1].obsm['spatial_aligned'].min().X]))
#     y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i - 1].obsm['spatial_aligned'].min().Y]))
#     adata_list[i].uns['grid_prop_withbefore'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
#                                                          grid_max=[x_max, y_max], grid_min=[x_min, y_min])

# # each slice is rasterized on the overlap of the next slice
# for i in range(0, len(adata_list)-1):
#     x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i + 1].obsm['spatial_aligned'].max().X]))
#     y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i + 1].obsm['spatial_aligned'].max().Y]))
#     x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i + 1].obsm['spatial_aligned'].min().X]))
#     y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i + 1].obsm['spatial_aligned'].min().Y]))
#     adata_list[i].uns['grid_prop_withnext'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
#                                                        grid_max=[x_max, y_max], grid_min=[x_min, y_min])

# # calculate SSIM on the overlap of each two adjacent slices
# for i in range(1, len(adata_list)):
#     grid_a = adata_list[i].uns['grid_prop_withbefore'].copy()
#     grid_b = adata_list[i - 1].uns['grid_prop_withnext'].copy()
#     col = set(grid_a.columns) & set(grid_b.columns)
#     grid_a = grid_a.loc[:, col]
#     grid_b = grid_b.loc[:, col]
#     ssim_list = []
#     for n in range(grid_a.shape[0]):
#         ssim_list.append(index.ssim(grid_a.iloc[n, :].values, grid_b.iloc[n, :].values))
#     ssim_list = np.array(ssim_list)
#     ssim_list[grid_b.sum(1) == 0] = np.inf
#     ssim_list[pd.isna(pd.Series(ssim_list))] = 0
#     adata_list[i].uns['ssim'] = ssim_list

# ssim_compare = []
# for i in range(1, len(adata_list)):
#     ssim_compare.append([adata_list[i].uns['ssim'][adata_list[i].uns['ssim'] != np.inf].mean(), i, 'STUtility'])
#     # ssim_compare.append([paste[i].uns['ssim'][paste[i].uns['ssim'] != np.inf].mean(), i, 'paste'])

# ssim_compare = pd.DataFrame(ssim_compare).fillna(0)
# ssim_compare.columns = ['ssim', 'slice', 'method']
# # ssim_compare.to_csv('G:/dataset/1_DLPFC/output/ssim_compare_donor1.csv')
# print(ssim_compare)

#### PCC

In [43]:
# import scipy
# groups = adatas.obs.groupby("batch").indices
# adata_list = [adatas[i] for i in groups.values()]

# for i in range(len(adata_list)):
#     adata_list[i].obsm['spatial_aligned'] = pd.DataFrame(adata_list[i].obsm['spatial_aligned'], index=adata_list[i].obs_names)
#     adata_list[i].obsm['spatial_aligned'].columns = ['X', 'Y']

# # each slice is rasterized on the overlap of the previous slice
# for i in range(1, len(adata_list)):
#     x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i - 1].obsm['spatial_aligned'].max().X]))
#     y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i - 1].obsm['spatial_aligned'].max().Y]))
#     x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i - 1].obsm['spatial_aligned'].min().X]))
#     y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i - 1].obsm['spatial_aligned'].min().Y]))
#     adata_list[i].uns['grid_prop_withbefore'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
#                                                          grid_max=[x_max, y_max], grid_min=[x_min, y_min])

# # each slice is rasterized on the overlap of the next slice
# for i in range(0, len(adata_list)-1):
#     x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i + 1].obsm['spatial_aligned'].max().X]))
#     y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i + 1].obsm['spatial_aligned'].max().Y]))
#     x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i + 1].obsm['spatial_aligned'].min().X]))
#     y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i + 1].obsm['spatial_aligned'].min().Y]))
#     adata_list[i].uns['grid_prop_withnext'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
#                                                        grid_max=[x_max, y_max], grid_min=[x_min, y_min])
    
# # calculate PCC on the overlap of each two adjacent slices
# for i in range(1, len(adata_list)):
#     grid_a = adata_list[i].uns['grid_prop_withbefore'].copy()
#     grid_b = adata_list[i - 1].uns['grid_prop_withnext'].copy()
#     col = set(grid_a.columns) & set(grid_b.columns)
#     grid_a = grid_a.loc[:, col]
#     grid_b = grid_b.loc[:, col]
#     pcc_list = []
#     for n in range(grid_a.shape[0]):
#         pcc_list.append(scipy.stats.pearsonr(grid_a.iloc[n, :].values, grid_b.iloc[n, :].values)[0])
#     pcc_list = np.array(pcc_list)
#     pcc_list[grid_b.sum(1) == 0] = np.inf
#     pcc_list[pd.isna(pd.Series(pcc_list))] = 0
#     adata_list[i].uns['pcc'] = pcc_list

# pcc_compare = []
# for i in range(1, len(adata_list)):
#     pcc_compare.append([adata_list[i].uns['pcc'][adata_list[i].uns['pcc'] != np.inf].mean(), i, 'STUtility'])
#     # ssim_compare.append([adata_list[i].uns['ssim'][adata_list[i].uns['ssim'] != np.inf].mean(), i, 'STUtility'])

# pcc_compare = pd.DataFrame(pcc_compare).fillna(0)
# pcc_compare.columns = ['pcc', 'slice', 'method']
# print(pcc_compare)
# # pcc_compare.to_csv('G:/dataset/1_DLPFC/output/pcc_compare_donor1.csv')

# Bio conservation

#### ARI

In [44]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']
methods = ['ground.truth']

for method in my_methods:
    methods.append('leiden_'+method)
    methods.append('louvain_'+method)
    methods.append('mclust_'+method)
print(methods)

for method in methods:
    print(scib.metrics.ari(adatas, method, 'ground.truth'))
    print(index.ARI(adatas, method, 'ground.truth'))

['ground.truth', 'leiden_SPIRAL_pyg', 'louvain_SPIRAL_pyg', 'mclust_SPIRAL_pyg']
1.0
1.0
0.5145526101811705
0.5145526101811705
0.5313750470593398
0.5313750470593398
0.4859487434869776
0.4859487434869776


#### NMI

In [45]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']
methods = ['ground.truth']

for method in my_methods:
    methods.append('leiden_'+method)
    methods.append('louvain_'+method)
    methods.append('mclust_'+method)
print(methods)

for method in methods:
    print(index.NMI(adatas, method, 'ground.truth'))

['ground.truth', 'leiden_SPIRAL_pyg', 'louvain_SPIRAL_pyg', 'mclust_SPIRAL_pyg']
1.0
0.6291757213952669
0.6546406993087601
0.6090790599013488


#### cASW

In [46]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(index.cASW(adatas, 'ground.truth', method+'_embedding'))
    print(index.cASW(adatas, 'leiden_'+method, method+'_embedding'))
    print(index.cASW(adatas, 'louvain_'+method, method+'_embedding'))
    print(index.cASW(adatas, 'mclust_'+method, method+'_embedding'))  

0.5396784065729743
0.5642388439993622
0.5519077695842152
0.5514091944803164


#### cLISI

In [47]:
# adatas = adatas.to_memory()

In [48]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(index.cLISI(adatas, 'ground.truth', type_="embed", use_rep=method+'_embedding'))
    print(index.cLISI(adatas, 'leiden_'+method, type_="embed", use_rep=method+'_embedding'))
    print(index.cLISI(adatas, 'louvain_'+method, type_="embed", use_rep=method+'_embedding'))
    print(index.cLISI(adatas, 'mclust_'+method, type_="embed", use_rep=method+'_embedding'))  

0.9171389573660216
0.9655644953907349
0.9541996851647495
0.9263462841931959


#### IsolatedLabelsAsw

In [49]:
import gc
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(index.IsolatedLabelsAsw(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding"))
    gc.collect()
    print(index.IsolatedLabelsAsw(adatas, batch_key="batch", label_key="leiden_"+method, embed=method+"_embedding"))
    gc.collect()
    print(index.IsolatedLabelsAsw(adatas, batch_key="batch", label_key="louvain_"+method, embed=method+"_embedding"))
    gc.collect()
    print(index.IsolatedLabelsAsw(adatas, batch_key="batch", label_key="mclust_"+method, embed=method+"_embedding"))
    gc.collect()
        
    


isolated labels: no more than 4 batches per label
Layer 1: 0.5138662086309655
Layer 2: 0.5506191115374636
Layer 3: 0.5005391295707124
Layer 4: 0.5260167910481086
Layer 5: 0.5301552121120908
Layer 6: 0.5041727227538995
WM: 0.6626943523458733
0.541151932571302
isolated labels: no more than 4 batches per label
0: 0.6107057709379108
1: 0.5863495801921368
2: 0.5599956471145846
3: 0.5214091145837589
4: 0.501750045179403
5: 0.5590315732854662
6: 0.6006754927547908
0.5628453177211501
isolated labels: no more than 4 batches per label
0: 0.5270109622781622
1: 0.5544735568749999
2: 0.5685115580493598
3: 0.6423351917687005
4: 0.506772567196484
5: 0.5531118299520246
6: 0.5191575898834797
0.553053322286173
isolated labels: no more than 4 batches per label
1: 0.5847882548405227
2: 0.6567941301283965
3: 0.535459556272697
4: 0.4953500139793494
5: 0.5436174202070622
6: 0.46422331946463885
7: 0.5759978109512184
0.5508900722634121


#### IsolatedLabelsF1

In [50]:
import gc

# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(method)
    print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding"))
    gc.collect()
    print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="leiden_"+method, embed=method+"_embedding"))
    gc.collect()
    print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="louvain_"+method, embed=method+"_embedding"))
    gc.collect()
    print(index.IsolatedLabelsF1(adatas, batch_key="batch", label_key="mclust_"+method, embed=method+"_embedding"))
    gc.collect()
        

SPIRAL_pyg
isolated labels: no more than 4 batches per label
Compute neighbors on rep SPIRAL_pyg_embedding
Layer 1: 0.8846002363135093
Compute neighbors on rep SPIRAL_pyg_embedding
Layer 2: 0.6987951807228915
Compute neighbors on rep SPIRAL_pyg_embedding
Layer 3: 0.8350237514443446
Compute neighbors on rep SPIRAL_pyg_embedding
Layer 4: 0.4328478964401295
Compute neighbors on rep SPIRAL_pyg_embedding
Layer 5: 0.8157365050320219
Compute neighbors on rep SPIRAL_pyg_embedding
Layer 6: 0.8096081905886986
Compute neighbors on rep SPIRAL_pyg_embedding
WM: 0.9001248439450689
0.7681052292123807
isolated labels: no more than 4 batches per label
Compute neighbors on rep SPIRAL_pyg_embedding
0: 0.996730632551528
Compute neighbors on rep SPIRAL_pyg_embedding
1: 0.9961345187475841
Compute neighbors on rep SPIRAL_pyg_embedding
2: 0.9928197166699011
Compute neighbors on rep SPIRAL_pyg_embedding
3: 0.9979568671963678
Compute neighbors on rep SPIRAL_pyg_embedding
4: 0.994459025955089
Compute neighbors o

# Batch correction

#### iASW

In [51]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'GraphST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(method)
    print(index.iASW(adatas, batch_key="batch", label_key="ground.truth", embed=method+"_embedding", type_="embed"))
    print(index.iASW(adatas, batch_key="batch", label_key="leiden_"+method, embed=method+"_embedding", type_="embed"))
    print(index.iASW(adatas, batch_key="batch", label_key="louvain_"+method, embed=method+"_embedding", type_="embed"))
    print(index.iASW(adatas, batch_key="batch", label_key="mclust_"+method, embed=method+"_embedding", type_="embed"))
    

SPIRAL_pyg
mean silhouette per group:          silhouette_score
group                    
Layer 1          0.965672
Layer 2          0.943932
Layer 3          0.942433
Layer 4          0.933950
Layer 5          0.944490
Layer 6          0.958410
WM               0.950067
0.9484221384072257
mean silhouette per group:        silhouette_score
group                  
0              0.965848
1              0.957036
2              0.949945
3              0.970261
4              0.951240
5              0.955549
6              0.907791
0.9510957859237621
mean silhouette per group:        silhouette_score
group                  
0              0.936565
1              0.949520
2              0.946415
3              0.959055
4              0.963511
5              0.951675
6              0.965480
0.9531746142601055
mean silhouette per group:        silhouette_score
group                  
1              0.974961
2              0.965291
3              0.966608
4              0.961306
5             

#### iLISI

In [52]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'GraphST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(method)
    print(index.iLISI(adatas, "batch", type="embed", use_rep=method+'_embedding'))

SPIRAL_pyg
0.6737916247589322


#### graph_connectivity

In [53]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'GraphST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(method)
    print(index.graph_connectivity(adatas, type_="embed", label_key='ground.truth'))
    print(index.graph_connectivity(adatas, type_="embed", label_key='leiden_'+method))
    print(index.graph_connectivity(adatas, type_="embed", label_key='louvain_'+method))
    print(index.graph_connectivity(adatas, type_="embed", label_key='mclust_'+method))
        


# print(index.graph_connectivity(adatas, type_="embed" ,label_key='ground.truth'))
# print(index.graph_connectivity(adatas, type_="embed" ,label_key='leiden_seurat'))
# print(index.graph_connectivity(adatas, type_="embed" ,label_key='louvain_seurat'))
# print(index.graph_connectivity(adatas, type_="embed" ,label_key='mclust_seurat'))

SPIRAL_pyg
0.9917550604108241
1.0
1.0
0.9924687742921837


#### kBET

In [10]:
# my_methods = ['seurat', 'harmony', 'LIGER', 'PRECAST', 'STAligner', 'DeepST', 'GraphST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(method)
    print(index.kBET(adatas, batch_key="batch", label_key='ground.truth', type_="embed", embed=method+"_embedding"))
    print(index.kBET(adatas, batch_key="batch", label_key='leiden_'+method, type_="embed", embed=method+"_embedding"))
    print(index.kBET(adatas, batch_key="batch", label_key='louvain_'+method, type_="embed", embed=method+"_embedding"))
    print(index.kBET(adatas, batch_key="batch", label_key='mclust_'+method, type_="embed", embed=method+"_embedding"))
        
        
# print(index.kBET(adatas, batch_key="batch", label_key="ground.truth", type_="embed", embed="seurat_embedding"))
# print(index.kBET(adatas, batch_key="batch", label_key="leiden_seurat", type_="embed", embed="seurat_embedding"))
# print(index.kBET(adatas, batch_key="batch", label_key="louvain_seurat", type_="embed", embed="seurat_embedding"))
# print(index.kBET(adatas, batch_key="batch", label_key="mclust_seurat", type_="embed", embed="seurat_embedding"))

seurat
Adding diffusion to step 4
Adding diffusion to step 5


D:\bio\metirc\scib-main\scib\metrics\kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
D:\bio\metirc\scib-main\scib\metrics\kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 6
Adding diffusion to step 7


D:\bio\metirc\scib-main\scib\metrics\kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


Adding diffusion to step 4
Adding diffusion to step 5


...

Adding diffusion to step 4


D:\bio\metirc\scib-main\scib\metrics\kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


Adding diffusion to step 4


D:\bio\metirc\scib-main\scib\metrics\kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


0.3747123865651695


#### PCR

In [55]:
# my_methods = ['GraphST', 'STAGATE', 'SPIRAL1024', 'SPIRAL2048']

for method in my_methods:
    print(method)
    print(index.PCR(adata_pre, adatas, covariate="batch", embed=method+"_embedding", n_comps=min(adatas.obsm[method+'_embedding'].shape[1], 100)))


# print(index.PCR(adata_pre, adatas, covariate="batch", embed="seurat_embedding", n_comps=adatas.obsm['seurat_embedding'].shape[1]))

# groups = adata_pre.obs.groupby("batch").indices
# adata_pre_list=[adata_pre[inds] for inds in groups.values()]
# print(adata_pre_list)

# groups = adatas.obs.groupby("batch").indices
# adatas_list=[adatas[inds] for inds in groups.values()]
# print(adatas_list)

# for i in range(4):
#     print(index.PCR(adata_pre_list[i], adatas_list[i], covariate="batch", embed="seurat_embedding", n_comps=adatas.obsm['seurat_embedding'].shape[1]))

SPIRAL_pyg


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:453: PendingDeprecationWarning: The dtype argument will be deprecated in anndata 0.10.0
  warnings.warn(


0.7925501974124514


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:453: PendingDeprecationWarning: The dtype argument will be deprecated in anndata 0.10.0
  warnings.warn(
